# Bending problem

In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.meshes import MakeStructured3DMesh
from utility_functions import get_polar_decomposition, Anti

import numpy as np
import time
import os
import matplotlib.pylab as plt

In [ ]:
use_autodiff = False
use_legacy = False
ngsglobals.code_uses_equivalence_keys=False # needs ngsolve 6.2.2601.post21.dev0 or newer
ngsglobals.code_uses_tensors=True
SetNumThreads(8)
draw_angle = { "euler_angles" : [20,-40,0] }


(Hint): Note that for multithreading,
```python
SetNumThreads(8)
```
only applies to the steps that NGSolve handles (like assembling). The actual solving is done by external libraries. Set the threads for that directly, e.g.:
```bash
export OMP_NUM_THREADS=8
export MKL_NUM_THREADS=8
export OPENBLAS_NUM_THREADS=8
```
on your machine.

In [ ]:
# --- Helper functions
def F(u):
    """
    Deformation gradient
    """
    return Grad(u) + Id(3)

def R(r):
    """
    Euler rotation matrix
    """
    r0 = r[0]
    r1 = r[1]
    r2 = r[2]
    
    cosr0 = cos(r0)
    cosr1 = cos(r1)
    cosr2 = cos(r2)
    
    sinr0 = sin(r0)
    sinr1 = sin(r1)
    sinr2 = sin(r2)
    return CF ( ( cosr1*cosr2, sinr0*sinr1*cosr2 - sinr2*cosr0,  sinr0*sinr2 + sinr1*cosr0*cosr2,
                sinr2*cosr1, sinr0*sinr1*sinr2 + cosr0*cosr2, -sinr0*cosr2 + sinr1*sinr2*cosr0,
                -sinr1     , sinr0*cosr1                    , cosr0*cosr1                      ), dims = (3,3) )

def CurlR(r):
    """
    Curl of Euler rotation matrix
    """
    r0 = r[0]
    r1 = r[1]
    r2 = r[2]
    
    cosr0 = cos(r0)
    cosr1 = cos(r1)
    cosr2 = cos(r2)
    
    sinr0 = sin(r0)
    sinr1 = sin(r1)
    sinr2 = sin(r2)
    
    Dr = Grad(r)
    Drx = CF( (Dr[0,0],Dr[0,1],Dr[0,2]), dims=(3,1) )
    Dry = CF( (Dr[1,0],Dr[1,1],Dr[1,2]), dims=(3,1) )
    Drz = CF( (Dr[2,0],Dr[2,1],Dr[2,2]), dims=(3,1) )
    
    dRz = CF( (sinr2*cosr1, sinr0*sinr1*sinr2 + cosr0*cosr2, -sinr0*cosr2 + sinr1*sinr2*cosr0, -cosr1*cosr2, -sinr0*sinr1*cosr2 + sinr2*cosr0, -sinr0*sinr2 - sinr1*cosr0*cosr2, 0, 0, 0), dims = (3,3) )
    dRy = CF( (sinr1*cosr2, -sinr0*cosr1*cosr2, -cosr0*cosr1*cosr2, sinr1*sinr2, -sinr0*sinr2*cosr1, -sinr2*cosr0*cosr1, cosr1, sinr0*sinr1, sinr1*cosr0), dims = (3,3) )
    dRx = CF( (0, -sinr0*sinr2 - sinr1*cosr0*cosr2, sinr0*sinr1*cosr2 - sinr2*cosr0, 0, sinr0*cosr2 - sinr1*sinr2*cosr0, sinr0*sinr1*sinr2 + cosr0*cosr2, 0, -cosr0*cosr1, sinr0*cosr1), dims = (3,3) )
    
    return CF(dRz * Anti(Drz) + dRy * Anti(Dry) + dRx * Anti(Drx))

In [ ]:
# --- Geometry
L = 20
b = 10
h = 1
n_refin = 1
mesh = MakeStructured3DMesh(
        hexes=False,
        nx=1 * (2**(n_refin)),
        ny=1 + (n_refin-1),
        nz=1* (2**(n_refin-1)),
        mapping=lambda x, y, z: (L*x,h*(y-1/2),b/2*(z)),
        secondorder=False
    )   
mesh.Refine()
Draw(mesh, **draw_angle)

In [ ]:
# --- Function spaces
# Displacement and microrotation:
order = 2
fesU = VectorH1(mesh, order=order, dirichlet="back", dirichletz="bottom")
fesR = VectorH1(mesh, order=order, dirichlet="back", dirichletx="bottom", dirichlety="bottom")
# fesR.SetOrder(TET, 4)
# fesR.Update()

fes = fesU * fesR
(u,r), (du,dr) = fes.TnT()

# Rotation tensor
fesZ = HCurl(mesh, order=1, type1=False, discontinuous=True)**3
RZ = Interpolate(R(r), fesZ, bonus_intorder=4)
polarRZ, _ = get_polar_decomposition(RZ,use_legacy=use_legacy, autodiff=use_autodiff)

In [ ]:
# --- Material parameters
E = Parameter( 1200 )
nu = Parameter( 0.3 )
lt = Parameter( 0.38 )
lb = Parameter( 0.5 )
psi = Parameter( 1.1 )
Lc = Parameter( 1 )
muc_factor = 1e4

mu = G = E/(2*(1+nu))
lam = - G*(E-2*G)/(E-3*G)
muc = muc_factor*G
alpha1 = 2*lt**2/Lc**2
alpha2 = (8*lb**2-2*lt**2)/Lc**2
alpha3 = 4*lt**2*(3-2*psi)/psi/Lc**2

# --- Elastic energy
def sym(A):
    """
    Use instead of Sym() of NGSolve to avoid problem with code_uses_equivalence_keys flag.
    """
    return 1/2 * (A + A.trans)

def skw(A):
    """
    Use instead of Skw() of NGSolve to avoid problem with code_uses_equivalence_keys flag.
    """
    return 1/2 * (A - A.trans)

def C(m):
    return lam * Trace(m) * Id(3) + 2*mu * m

def PSI(U, A):
    return 1/2 * (
            InnerProduct(sym(U) - Id(3), C(sym(U) - Id(3)))
        + 2*muc*InnerProduct(skw(U), skw(U))
        + mu*Lc**2 * (
            alpha1*InnerProduct(Deviator(sym(A)),Deviator(sym(A)))
            + alpha2*InnerProduct(skw(A),skw(A))
            + alpha3/3 * Trace(A)**2
        )
        #+ mu*Lc**2 * (InnerProduct(A,A))
        )

In [ ]:
# --- Strain measures
# Stretch tensor U=R.trans*F
U = CF(R(r).trans*F(u))
#U = CF(RZ.trans*F(u))
#U = CF(polarRZ.trans*F(u))

# Dislocation tensor A = R.trans*Curl(R)
A = CF(R(r).trans*CurlR(r))

# --- Set up bilinear form
a = BilinearForm(fes)
ts = time.time()
#a += Variation ( PSI(U, A).Compile(realcompile=True, wait=True, keep_files=True) * dx )
a += Variation ( PSI(U, A).Compile() * dx )
te = time.time()
print (f"Compile took {te-ts} seconds")

# Set up load
force = 40
loadfactor = Parameter(1)
traction = loadfactor * force / b / h / 2 * CF((-1,1,0))
a += - (traction*du)*ds("front")

In [ ]:
# --- Define grid function
gfu = GridFunction(fes)
print(f"There are {gfu.space.ndof} DoFs")

In [ ]:
# --- Solve problem with adaptive load stepping
steps = 40
max_load = 1.0
loadvalue = 0.0
load_step = 1.0 / steps
min_load_step = 1e-5
max_load_step = 0.5
reduce_factor = 0.5
increase_factor = 1.1

scene = Draw(gfu.components[0], deformation=True, **draw_angle)
step = 0
while loadvalue < max_load - 1e-12:
    try:
        loadvalue_next = min(loadvalue + load_step, max_load)
        loadfactor.Set(loadvalue_next)

        print(f" --- Step {step+1}: target load {loadvalue_next:.6f} --- ", flush=True)

        old_gfu = gfu.vec.CreateVector()  # deep copy to restore on failure
        old_gfu.data = gfu.vec

        with TaskManager():
            start = time.perf_counter()
            converged, newton_iterations = solvers.Newton(a, gfu, maxerr=1e-9,
                                                        printing=True, maxit=15, dampfactor=1)
            end = time.perf_counter()

        if converged == -1:
            raise RuntimeError("Computation did not converge")

        print(f"Execution time: {end - start:.3f}s with {newton_iterations} Newton iterations",
            flush=True)
        scene.Redraw()
        # Step succeeded - accept and maybe enlarge next step
        loadvalue = loadvalue_next
        step += 1
        load_step = min(load_step * increase_factor, max_load_step)
    except Exception as e:
        print(f"Step {step+1} failed (load={loadvalue_next:.6f}). Reducing step...", flush=True)
        gfu.vec.data = old_gfu  # restore previous converged state
        load_step *= reduce_factor
        if load_step < min_load_step:
            print("Load step became too small — stopping.")
            break

In [ ]:
# Extract solution from gridfunction
u_sol = gfu.components[0]
r_sol = gfu.components[1]
F_sol = F(u_sol)

# --- Define name of test:
test_name = "bending"
os.makedirs("results", exist_ok=True)
filename = os.path.join("results", test_name)
vtk = VTKOutput(mesh,coefs=[u_sol, r_sol, F_sol],names=["u","r","F"],filename=os.path.join("results", "bending"),subdivision=0)
vtk.Do()